Sentiment Analysis - positive or Negative => binary classification problem
IMDb -- Movie reviews dataset
Recurrent NN-- Process sequences of data --NlP
* time series
* Text in sentences
recurrent -- the NN contains loopss - output of a given layer becomes the input to the same layer in the next word in aa sequence of words for text sequence
* loops in RNNs help them to learn relationships among the data in the sequence.
Good -- on its own is a posivtive sentiment
Not good has a negative sentiment,, Not is earlier in the seauence
RNN do consider the relationships among earlier and later data in the sequence. here the words that determine sentiment are adjacent. But whn considering text's meaning there can  be many words to consider and abitary  nunmber of words between them.

Long short- Term Memory (LSTM) -- layer that makes the neural network recurrent.

- Predictive text input
- Sentiment Analysis
- Responding to questions with predicted best answers
- Inter- language translation
- automated video closed captioning -- speech recognition
- speech synthesis

#Loading the IMDb Movie Review Dataset

In [ ]:
from tensorflow.keras.datasets import imdb

88,000  unique words in a dataset
* you can specifyy the number of unique words to import -- training and testing data
* usually 10,000 most frquent occuring words are used
* Due to systemmemeory limitations and training on cpu -- GPU,, TPU
-- the more data the longer it takes to train-- it may produce better models

In [ ]:
#load_data  replaces any words outside the  10,000 with a placeholder
number_of_words=10000

In [ ]:
import tensorflow as tf

In [ ]:
import numpy as np

In [ ]:
#save the np.load
np_load_old=np.load

In [ ]:
#modify the defalut parameters of np.load
# *a is a list , **k is an object -- argc
np.load=lambda *a,**k: np_load_old(*a, allow_pickle=True)

In [ ]:
(x_train,y_train),(x_test,y_test)= imdb.load_data(num_words=number_of_words)

In [ ]:
#x-test and x_xtrain apear as 1 dimentional -- Numpy array of objects(list of intergers)

In [ ]:
x_train.shape

(25000,)

In [ ]:
x_test.shape

(25000,)

In [ ]:
y_train.shape

(25000,)

In [ ]:
y_test.shape

(25000,)

In [ ]:
y_train

array([1, 0, 0, ..., 0, 1, 0])

# Data Exploration

The arrays y_train and y_test -1D with 0s(negative) and 1s(postive). the x_train and x_test are list og intergers each represenig one reviews's contents.
Keras models require numeric data -- IMDb data is preprocessed for you

In [ ]:
%pprint # toggle better printing so thats elements do not display vertically

Pretty printing has been turned OFF


In [ ]:
x_train[123]

[1, 307, 5, 1301, 20, 1026, 2511, 87, 2775, 52, 116, 5, 31, 7, 4, 91, 1220, 102, 13, 28, 110, 11, 6, 137, 13, 115, 219, 141, 35, 221, 956, 54, 13, 16, 11, 2714, 61, 322, 423, 12, 38, 76, 59, 1803, 72, 8, 2, 23, 5, 967, 12, 38, 85, 62, 358, 99]

to view the original text you need to know the word to which each number corresponds to.
The keras dictionary - provides the dictionary that maps words to their indexes.
EACH WORDS VALUE IS ITS FREQUENCY OCCURING WORD ... FOR Example randing 1 -- frequently occuring in that dataset
-Training and the testing data -- have an offset of3. (1+3) = 4-- most frequently occuring word has a value of 4.
the values 0,1,2 are reserved
adding (0) -- all the traing /testingmusut have the sam dimentions
 some reviews may nee d to be padded with a 0 and some shorted
 start of the sequnce (1) -- a token used by keras internally for learnig purposes

 unknown word(2) not loaded -- load_data uses 2 for words with freq ranks > than the num-words

In [ ]:
word_to_index=imdb.get_word_index()

In [ ]:
word_to_index['great']

84

In [ ]:
#great is the 84th most freq word

In [ ]:
#lets
index_to_word={index: word for (word, index)in word_to_index.items()}

In [ ]:
[index_to_word[i] for i in range(1,51)]

['the', 'and', 'a', 'of', 'to', 'is', 'br', 'in', 'it', 'i', 'this', 'that', 'was', 'as', 'for', 'with', 'movie', 'but', 'film', 'on', 'not', 'you', 'are', 'his', 'have', 'he', 'be', 'one', 'all', 'at', 'by', 'an', 'they', 'who', 'so', 'from', 'like', 'her', 'or', 'just', 'about', "it's", 'out', 'has', 'if', 'some', 'there', 'what', 'good', 'more']

# Decoding a Movie Review

- To decode a review
    - i-3 accounts for the frequency ratings offsets in the encoded reviews
    - i values 0-2 = ? otherwise should return the word

In [ ]:
' '.join([index_to_word.get(i-3, '?') for i in x_train[123]])

'? beautiful and touching movie rich colors great settings good acting and one of the most charming movies i have seen in a while i never saw such an interesting setting when i was in china my wife liked it so much she asked me to ? on and rate it so other would enjoy too'

In [ ]:
y_train[123]

1

# Data Preparation

- Number of words per review varies
- Keras requires all samples to have the same dimensions
- Prepare the data
    - Restrict every review to the same number of words
    - Pad some with 0s, truncate others
    - pad-sequences function - reshapes the samples and returns a 2D array

In [ ]:
words_per_review = 200

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
x_train = pad_sequences(x_train, maxlen = words_per_review)

In [ ]:
x_train.shape

(25000, 200)

In [ ]:
x_train

array([[   5,   25,  100, ...,   19,  178,   32],
       [   0,    0,    0, ...,   16,  145,   95],
       [   0,    0,    0, ...,    7,  129,  113],
       ...,
       [   0,    0,    0, ...,    4, 3586,    2],
       [   0,    0,    0, ...,   12,    9,   23],
       [   0,    0,    0, ...,  204,  131,    9]], dtype=int32)

# Data Preparation - Cont.

- Reshape the x_test

In [ ]:
x_test=pad_sequences(x_test, maxlen = words_per_review)

In [ ]:
x_test.shape

(25000, 200)

## Splitting the test data into validation and test data

    - Split the 25,000 test samples into 20,000 test samples and 5000 validation samples
    - We pass vaalidation samples to the models fit method - validation_data argument
        - use the scikit-learn train_test_split function

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_test,x_val,y_test,y_val=train_test_split(x_test,y_test,random_state=11, test_size=0.20)


In [ ]:
x_test.shape

(20000, 200)

In [ ]:
x_val.shape

(5000, 200)

# Creating the Neural Network

In [ ]:
from tensorflow.keras.models import Sequential

In [ ]:
rnn = Sequential()

In [ ]:
rnn

In [ ]:
from tensorflow.keras.layers import Dense, LSTM, Embedding

### Adding an Embedding Layer

- In our convne example we used one-hot encoding - into categorical data - others 0 and one 1
- index values that represent words
- 1, 00000000000000000000000000000000000000000000000000000000000000000000000
    - 0 100000000000000000000000000000000000000000000000000000000000000000
- recall cnn.add(Dense(units=10, activation ='softmax')) - 0,1,2,3,4,5,6,7,8,9 => 0.0003, 0.88...
- Will need 10,000 by 10,000 array to represent all the words
- 100,000,000 array to represent the words
- All would be 0 except 1
- all 88,000 +unique words -- 8billion elements

- Reduce dimensionality - RNN process Text sequences, use the embedding layer
- Each encodes each word in a more compact dense-vector representation
- Help RNN - word relations -> this movie is not good --0 this movie is good -- 1
    * Predefined word embeddings -- Word2Vec and GloVe
        - can load into nn to save training time
        - used to add basic word relationships to model smaller amounts of training  data (available)
        - Improve model accuracy - by looking at the previous word relations -Transfer Learning

In [ ]:
rnn.add(Embedding(input_dim = number_of_words, output_dim = 128, input_length = words_per_review))

- input_dim === Number of unique words
- output_dim-- six of the word embedding
- input_length== Number of words in each input sample

In [ ]:
# Adding an LSTM Layer
rnn.add(LSTM(units = 128, dropout = 0.2, recurrent_dropout=0.2))

- Units -- number of neuronsin that layer (the more the neurons the more the network can remember)
- length of the sequence - 200 -- and the number of classes to predict -- 2 class - 2-200
- dropout- percentage of neurons that are disabled when processing the layer's input and output

## Adding Dense Output Layer

- We need to reduce the LTSM layer's output to 1 value (result should be one)-

+ or - We need te value of 1 for the units arguments
- sigmoid activation function - used for bnary classification - reduces arbitrary values into the range 0.0-1.-, producing a probability

In [ ]:
rnn.add(Dense(units=1, activation='sigmoid'))

#Compiling the model and display the summary
- 'categorical_crossentropy'--'binary_crossentropy'regresssion - 'mean_squared_error'

In [ ]:
rnn.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
rnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 128)          1280000   
_________________________________________________________________
lstm (LSTM)                  (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 1,411,713
Trainable params: 1,411,713
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#Training and Evaluating the model

In [ ]:
x_train.shape

(25000, 200)

-For every Epoch the RNN model takes longer ti train than the CNN (Number of parameters)

In [ ]:
rnn.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test,y_test))


Epoch 1/10
782/782 [==============================] - 479s 613ms/step - loss: 0.3628 - accuracy: 0.8463 - val_loss: 0.3052 - val_accuracy: 0.8741
Epoch 2/10
782/782 [==============================] - 489s 626ms/step - loss: 0.2376 - accuracy: 0.9087 - val_loss: 0.3346 - val_accuracy: 0.8696
Epoch 3/10
782/782 [==============================] - 487s 623ms/step - loss: 0.1806 - accuracy: 0.9325 - val_loss: 0.5050 - val_accuracy: 0.8394
Epoch 4/10
782/782 [==============================] - 480s 613ms/step - loss: 0.1367 - accuracy: 0.9491 - val_loss: 0.4674 - val_accuracy: 0.8363
Epoch 5/10
782/782 [==============================] - 481s 615ms/step - loss: 0.1100 - accuracy: 0.9604 - val_loss: 0.4259 - val_accuracy: 0.8605
Epoch 6/10
782/782 [==============================] - 485s 620ms/step - loss: 0.0885 - accuracy: 0.9680 - val_loss: 0.4980 - val_accuracy: 0.8543
Epoch 7/10
782/782 [==============================] - 488s 624ms/step - loss: 0.0681 - accuracy: 0.9776 - val_loss: 0.5684 -

In [ ]:
results=rnn.evaluate(x_test, y_test)
results

#Tuning Deep Learning Models
Some variables that can affect the model performance
- more or less data to train with
- more or less data to test with
- having more or fewer layers
- types of layers you use
- order of the layers

-Some things that we could tune on our model:
- Try differernt amounts of training data- 10,000
- Different number of words per review - 200,  300
- Different number of neurons in our layers
- More layers
-  Transfer learning- loading pre-trained word vectors rather learning from scratch
* k-fold cross-validation - 10 -fold cross validation ANN - longer time - tuning hyperparameters

** Auto-keras